# Отбор и селекция признаков

## Способ представления данных

In [319]:
#Импортируем необходимые библиотеки

#Для матричных вычислений
import numpy as np
#Для анализа и предобработки данных
import pandas as pd
#Для визуализации
import matplotlib.pyplot as plt
import seaborn as sns

#Метрики
from sklearn import metrics
#Методы разделения и валидации
from sklearn import model_selection
#Ансамбли
from sklearn import ensemble
#Линейные модели
from sklearn import linear_model

#Стиль отрисовки seaborn
plt.style.use('seaborn-v0_8')
%matplotlib inline

In [320]:
#Загрузим данные
data = pd.read_excel('data/data_ford_price.xlsx')
display(data.head())

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,1300,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN


In [321]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         7017 non-null   int64  
 1   year          7017 non-null   int64  
 2   condition     7017 non-null   int64  
 3   cylinders     7017 non-null   int64  
 4   odometer      7017 non-null   int64  
 5   title_status  7017 non-null   object 
 6   transmission  7017 non-null   object 
 7   drive         6626 non-null   object 
 8   size          5453 non-null   object 
 9   lat           7017 non-null   float64
 10  long          7017 non-null   float64
 11  weather       6837 non-null   float64
dtypes: float64(3), int64(5), object(4)
memory usage: 658.0+ KB


## Кодирование признаков

Обратимся к нашим данным. Нас просят предсказать цену автомобиля. Для обучения модели выделим целевой столбец:

In [322]:
y = data['price']
X = data.drop(columns='price')

Попробуем применить линейную регрессию на «сырых» данных:

In [323]:
# lr = linear_model.LinearRegression()
# lr.fit(X, y)

Мы получим ошибку с комментарием о том, что не удалось превратить строковое значение в число с плавающей точкой (float).

Давайте посмотрим на кодирование признака Образование способом «один-против-всех» (one vs all):

In [324]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()

education = ['нет', 'начальное', 'среднее', 'BSc', 'MSc', 'начальное', 'PhD']

lb.fit(education)

print('категории:', lb.classes_)

lb.transform(['нет', 'MSc'])

категории: ['BSc' 'MSc' 'PhD' 'начальное' 'нет' 'среднее']


array([[0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0]])

Посмотрим на число уникальных значений номинальных признаков title_status, transmission, drive, size и cylinders:

In [325]:
columns_to_change = ['cylinders', 'title_status', 'transmission',
                     'drive', 'size']

for column in columns_to_change:
    print('Число уникальных значений признака {}:'.format(column), data[column].nunique())

Число уникальных значений признака cylinders: 6
Число уникальных значений признака title_status: 5
Число уникальных значений признака transmission: 3
Число уникальных значений признака drive: 3
Число уникальных значений признака size: 4


Итак, нам подходит однократное кодирование. Применим его к выбранным столбцам. Так как у нас нет отдельной тестовой выборки, то мы используем только один метод — fit_transform(). В качестве аргумента передаём таблицу с выбранными для преобразования признаками.

С помощью метода get_feature_names_out() получим список новых названий колонок:

In [326]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder()

#'Учим' и сразу применяем преобразование к выборке, 
#результат переводим в массив
data_onehot = one_hot_encoder.fit_transform(data[columns_to_change]).toarray()

#Запишем полученные названия новых колонок в отдельную переменную
column_names = one_hot_encoder.get_feature_names_out(columns_to_change)
print(column_names)

['cylinders_3' 'cylinders_4' 'cylinders_5' 'cylinders_6' 'cylinders_8'
 'cylinders_10' 'title_status_clean' 'title_status_lien'
 'title_status_missing' 'title_status_rebuilt' 'title_status_salvage'
 'transmission_automatic' 'transmission_manual' 'transmission_other'
 'drive_4wd' 'drive_fwd' 'drive_rwd' 'drive_nan' 'size_compact'
 'size_full-size' 'size_mid-size' 'size_sub-compact' 'size_nan']


### Задание 3.7

In [327]:
data_onehot = pd.DataFrame(data_onehot, index=data.index, columns=column_names)
display(data_onehot)

,cylinders_3,cylinders_4,cylinders_5,cylinders_6,cylinders_8,cylinders_10,title_status_clean,title_status_lien,title_status_missing,title_status_rebuilt,...,transmission_other,drive_4wd,drive_fwd,drive_rwd,drive_nan,size_compact,size_full-size,size_mid-size,size_sub-compact,size_nan
0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7012,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
7013,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
7014,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
7015,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [328]:
data_new = pd.concat([data, data_onehot], axis=1)

In [329]:
data_new = data_new.drop(columns=columns_to_change)

In [330]:
display(data_new.shape)

(7017, 30)

## Обработка пропусков и выбросов

In [331]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         7017 non-null   int64  
 1   year          7017 non-null   int64  
 2   condition     7017 non-null   int64  
 3   cylinders     7017 non-null   int64  
 4   odometer      7017 non-null   int64  
 5   title_status  7017 non-null   object 
 6   transmission  7017 non-null   object 
 7   drive         6626 non-null   object 
 8   size          5453 non-null   object 
 9   lat           7017 non-null   float64
 10  long          7017 non-null   float64
 11  weather       6837 non-null   float64
dtypes: float64(3), int64(5), object(4)
memory usage: 658.0+ KB


None

In [332]:
#Посмотрим данные на пропуски в более удобном формате
data.isnull().sum()

price              0
year               0
condition          0
cylinders          0
odometer           0
title_status       0
transmission       0
drive            391
size            1564
lat                0
long               0
weather          180
dtype: int64

Посмотрим, что будет, если просто убрать все строки с пропусками в столбце weather:

In [333]:
data[~data['weather'].isna()]

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,1300,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
5,6995,2003,3,8,167662,clean,automatic,4wd,full-size,45.518031,-122.578752,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7012,22500,2015,3,6,23500,clean,automatic,rwd,full-size,32.680700,-117.169800,59.0
7013,5975,2005,2,8,0,clean,automatic,rwd,full-size,38.213303,-85.785762,50.0
7014,9999,2006,3,8,161514,clean,automatic,NaN,full-size,37.609783,-120.995406,59.0
7015,10900,2011,2,8,164000,clean,automatic,4wd,full-size,43.140600,-93.385000,47.0


### Задание 4.2

Какая доля строк в датасете останется, если убрать пропуски в столбце size? Ответ округлите до двух знаков после точки-разделителя.

In [334]:
round(data[~data['size'].isna()].shape[0] / data.shape[0], 2)

0.78

В качестве регрессора воспользуемся линейной моделью, а качество оценим с помощью коэффициента детерминации. Также нам потребуется разделить модель на обучающую и тестовую выборки.

In [335]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

Удалим данные с пропусками:

In [336]:
X = X.dropna()

Выберем все оставшиеся индексы таблицы x с помощью метода index(), а затем используем .iloc[], чтобы получить подгруппу целевых значений, соответствующую полученным индексам.

In [337]:
y = y.iloc[X.index]

Проверьте, что переменные x и y имеют одинаковую длину.

In [338]:
X.shape[0] == y.shape[0]

True

Разделим выборку на тренировочную и тестовую в соотношении 80/20:

In [339]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

Проведём кодирование OneHot-методом категориальных переменных.

In [340]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder()

Обучаем энкодер и сразу применяем преобразование к выборке. Результат переводим в массив:

In [341]:
X_train_onehot = one_hot_encoder.fit_transform(X_train[columns_to_change]).toarray()

Затем применяем полученное преобразование к тестовой выборке. Результат переводим в массив:

In [342]:
X_test_onehot = one_hot_encoder.transform(X_test[columns_to_change]).toarray()

Для удобства сохраним полученные названия новых колонок в отдельную переменную:

In [343]:
columns = one_hot_encoder.get_feature_names_out(columns_to_change)
columns[:10]

array(['cylinders_3', 'cylinders_4', 'cylinders_5', 'cylinders_6',
       'cylinders_8', 'cylinders_10', 'title_status_clean',
       'title_status_lien', 'title_status_missing',
       'title_status_rebuilt'], dtype=object)

Теперь у нас есть массив закодированных признаков и наша изначальная таблица. Чтобы соединить эти данные, переведём массив в формат DataFrame.

In [344]:
X_train_onehot_df = pd.DataFrame(X_train_onehot, columns=columns)
X_test_onehot_df = pd.DataFrame(X_test_onehot, columns=columns)

Переустановим индексацию в таблицах, применив подряд сразу два метода: reset_index() — для изменения индексов с рандомных на последовательные от 0 до n и drop(['index'], axis = 1) — для удаления образовавшегося столбца 'index'.

In [345]:
X_train = X_train.reset_index().drop(['index'], axis=1)
X_test = X_test.reset_index().drop(['index'], axis=1)

y_train = y_train.reset_index().drop(['index'], axis=1)
y_test = y_test.reset_index().drop(['index'], axis=1)

Объединяем таблицы и удаляем старые категориальные признаки:

In [346]:
X_train_new = pd.concat([X_train, X_train_onehot_df], axis=1)
X_test_new = pd.concat([X_test, X_test_onehot_df], axis=1)

X_train_new = X_train_new.drop(columns=columns_to_change)
X_test_new = X_test_new.drop(columns=columns_to_change)

Настало время обучить модель. Для этого создаём объект класса LinearRegression.

In [347]:
lr_model = LinearRegression()

In [348]:
#Обучаем модель
lr_model.fit(X_train_new, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [349]:
#Делаем предсказание для тренировочной выборки
y_train_predict = lr_model.predict(X_train_new)
#Делаем предсказание для тествой выборки
y_test_predict = lr_model.predict(X_test_new)

print('Train r2 score: {:.3f}'.format(r2_score(y_train, y_train_predict)))
print('Test r2 score: {:.3f}'.format(r2_score(y_test, y_test_predict)))

Train r2 score: 0.647
Test r2 score: 0.693


→ Теперь давайте попробуем заполнить пропуски константными значениями и обучить модель заново. Плюс такого подхода состоит в том, что мы предотвращаем потерю данных, которая происходит при удалении строк или столбцов. Основной минус — в снижении разброса (разнообразия) признаков.

Заполним числовой столбец средним значением, округлив его до целого числа:

In [350]:
import numpy as np

X_train['weather'] = X_train['weather'].fillna(np.round(np.mean(X_train['weather']), 0))
X_test['weather'] = X_test['weather'].fillna(np.round(np.mean(X_test['weather']), 0))

Для простоты воспользуемся заполнением наиболее частым значением категориальных признаков. Для этого сначала определим их в наших признаках, использовав комбинацию методов value_counts() и head():

In [351]:
X_train['drive'].value_counts(True).head(1)

drive
4wd    0.736602
Name: proportion, dtype: float64

In [352]:
X_train['size'].value_counts(True).head(1)

size
full-size    0.830089
Name: proportion, dtype: float64

In [353]:
X_train['size'] = X_train['size'].fillna('full-size')
X_train['drive'] = X_train['drive'].fillna('4wd')

X_test['size'] = X_test['size'].fillna('full-size')
X_test['drive'] = X_test['drive'].fillna('4wd')

После обучения модели получился следующий результат:

In [354]:
data = pd.read_excel('data/data_ford_price.xlsx')

y = data['price']
X = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

import numpy as np

X_train['weather'] = X_train['weather'].fillna(np.round(np.mean(X_train['weather']), 0))
X_test['weather'] = X_test['weather'].fillna(np.round(np.mean(X_test['weather']), 0))

X_train['size'] = X_train['size'].fillna('full-size')
X_train['drive'] = X_train['drive'].fillna('4wd')

X_test['size'] = X_test['size'].fillna('full-size')
X_test['drive'] = X_test['drive'].fillna('4wd')

from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder()

X_train_onehot = one_hot_encoder.fit_transform(X_train[columns_to_change]).toarray()

X_test_onehot = one_hot_encoder.transform(X_test[columns_to_change]).toarray()

columns = one_hot_encoder.get_feature_names_out(columns_to_change)

X_train_onehot_df = pd.DataFrame(X_train_onehot, columns=columns)
X_test_onehot_df = pd.DataFrame(X_test_onehot, columns=columns)

X_train = X_train.reset_index().drop(['index'], axis=1)
X_test = X_test.reset_index().drop(['index'], axis=1)

y_train = y_train.reset_index().drop(['index'], axis=1)
y_test = y_test.reset_index().drop(['index'], axis=1)

X_train_new = pd.concat([X_train, X_train_onehot_df], axis=1)
X_test_new = pd.concat([X_test, X_test_onehot_df], axis=1)

X_train_new = X_train_new.drop(columns=columns_to_change)
X_test_new = X_test_new.drop(columns=columns_to_change)



lr_model = LinearRegression()

#Обучаем модель
lr_model.fit(X_train_new, y_train)

#Делаем предсказание для тренировочной выборки
y_train_predict = lr_model.predict(X_train_new)
#Делаем предсказание для тествой выборки
y_test_predict = lr_model.predict(X_test_new)

print('Train r2 score: {:.3f}'.format(r2_score(y_train, y_train_predict)))
print('Test r2 score: {:.3f}'.format(r2_score(y_test, y_test_predict)))


Train r2 score: 0.649
Test r2 score: 0.465


Приведённые методы обработки отсутствующих значений не учитывают корреляционную связь признака, содержащего пропуски, с остальными. Признаки, не имеющие NaN, можно использовать для прогнозирования пропущенных значений. Строится модель регрессии или классификации в зависимости от характера (категорийного или непрерывного) признака, имеющего пропущенное значение.

In [355]:
#Импортируем необходимые модули
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import OneHotEncoder

#Скопируем данные в отдельную переменную

copy_X = X.copy()

#В качестве тестовой выборки возьмем строки с пропусками в признаке weather
test_copy_X = copy_X[copy_X['weather'].isnull()]
#И удалим эти срочки из таблицы
copy_X.dropna(inplace=True)

#Определим целевой признак и факторы
y_train = copy_X['weather']
X_train = copy_X.drop(['size', 'weather', 'drive'], axis=1)
X_test = test_copy_X.drop(['size', 'weather', 'drive'], axis=1)

#Создадим кодировщик 
one_hot_encoder = OneHotEncoder()
categorial_cols = ['cylinders', 'title_status', 'transmission']

#Закодируем категориальные признаки 
#(обучаем кодировщик только на тренировочной выборке)
X_train_onehot = one_hot_encoder.fit_transform(X_train[categorial_cols]).toarray()
X_test_onehot = one_hot_encoder.transform(X_test[categorial_cols]).toarray()

#Результаты преобразуем обратно в DF для удобства
columns = one_hot_encoder.get_feature_names_out(categorial_cols)
X_train_onehot_df = pd.DataFrame(X_train_onehot, columns=columns)
X_test_onehot_df = pd.DataFrame(X_test_onehot, columns=columns)

#Сбросим индексы таблиц
X_train = X_train.reset_index().drop(['index'], axis=1)
X_test = X_test.reset_index().drop(['index'], axis=1)
y_train = y_train.reset_index().drop(['index'], axis=1)

#Добавим результаты кодирования к исходным таблицам 
X_train_new = pd.concat([X_train, X_train_onehot_df], axis=1)
X_test_new = pd.concat([X_test, X_test_onehot_df], axis=1)

#Удалим столбцы, которые уже были закодированы
X_train_new = X_train_new.drop(columns=categorial_cols)
X_test_new = X_test_new.drop(columns=categorial_cols)

#Создадим модель линейной регрессии и обучим ее на задачу предсказания пропусков
model = LinearRegression()
model.fit(X_train_new, y_train)

#Сделаем предсказание целевой переменной (пропущенных значений в признаке weather)
y_pred = model.predict(X_test_new)

display(y_pred[:10])


array([[40.91435555],
       [40.7637233 ],
       [39.74866152],
       [41.2755305 ],
       [40.31791932],
       [41.10796547],
       [41.15337846],
       [39.94866488],
       [41.10796547],
       [40.7217165 ]])

### Задание 4.5

In [356]:
for i, ni in enumerate(test_copy_X.index):
    X.loc[ni, 'weather'] = y_pred[i]

X.isnull().sum()

year               0
condition          0
cylinders          0
odometer           0
title_status       0
transmission       0
drive            391
size            1564
lat                0
long               0
weather            0
dtype: int64

In [357]:
def encode_cat_features(columns_to_change, X_train, X_test, y_train):
    one_hot_encoder = OneHotEncoder()
    X_train_onehot = one_hot_encoder.fit_transform(X_train[columns_to_change]).toarray()
    X_test_onehot = one_hot_encoder.transform(X_test[columns_to_change]).toarray()

    columns = one_hot_encoder.get_feature_names_out(columns_to_change)
    
    X_train_onehot_df = pd.DataFrame(X_train_onehot, columns=columns)
    X_test_onehot_df = pd.DataFrame(X_test_onehot, columns=columns)

    X_train = X_train.reset_index().drop(['index'], axis = 1)
    X_test = X_test.reset_index().drop(['index'], axis = 1)
    y_train = y_train.reset_index().drop(['index'], axis = 1)

    X_train_new = pd.concat([X_train, X_train_onehot_df], axis=1)
    X_test_new = pd.concat([X_test, X_test_onehot_df], axis=1)
    
    X_train_new = X_train_new.drop(columns=columns_to_change)
    X_test_new = X_test_new.drop(columns=columns_to_change)

    return X_train_new, X_test_new

Более простой вариант решения

In [359]:
#Скопируем данные в отдельную переменную
copy_X = X.copy()
#В качестве тестовой выборки возьмем строки с пропусками в признаке weather
test_copy_X = copy_X[copy_X['size'].isnull()]
#И удалим эти срочки из таблицы
copy_X.dropna(inplace=True)

#Определим целевой признак и факторы
y_train = copy_X['size']
X_train = copy_X.drop(['size', 'drive'], axis=1)
X_test = test_copy_X.drop(['size', 'drive'], axis=1)

X_train_new, X_test_new = encode_cat_features(categorial_cols, X_train, X_test, y_train)

In [360]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_new, y_train)
 
y_pred = model.predict(X_test_new)
y_pred

c:\Program Files (x86)\Python\Lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array(['full-size', 'full-size', 'full-size', ..., 'full-size',
       'full-size', 'full-size'], dtype=object)

In [361]:
for i, ni in enumerate(test_copy_X.index):
    X.loc[ni, 'size'] = y_pred[i]

X.isnull().sum()

year              0
condition         0
cylinders         0
odometer          0
title_status      0
transmission      0
drive           391
size              0
lat               0
long              0
weather           0
dtype: int64

### Работа с выбросами